[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/taylor-SUNY-Poly/CS295_Public/blob/main/lunar_lander/lunar_lander_colab_port.ipynb)

# Lunar Lander NEAT — Colab Runtime

This notebook sets up a Google Colab runtime for the Lunar Lander neuroevolution lab. It mirrors the command-line workflow that students follow locally, but leverages Colab's compute resources.

**Workflow overview**

1. Install system and Python dependencies required by Gymnasium's Box2D environment and the NEAT lab code.
2. Ensure the working directory is this `colab_port/` folder so generated artefacts match the local layout.
3. Launch training/evaluation commands exactly as described in the lab handout (`python lunarlander.py <phase>`).
4. Periodically download the `phase*/` folders as a ZIP so progress is safe outside the ephemeral Colab VM.

Refer to the accompanying [README](README.md) for best practices when running extended jobs on Colab.

## 1. Install runtime dependencies

Run the cell below once per Colab session. It installs the system package **swig** (needed by `gymnasium[box2d]`) and the Python libraries referenced by `lunarlander.py` and `report_utils.py`.

In [ ]:
%%bash
set -e
apt-get update -qq
apt-get install -y swig > /dev/null

In [ ]:
!pip install --quiet neat-python "gymnasium[box2d]" imageio matplotlib

## 2. Set the working directory

When this notebook is opened via "Open in Colab" from GitHub, the repository is cloned into `/content`. The helper cell below switches into the `colab_port/` folder so any `phase*/` directories, configs, and genomes are written next to the scripts. Adjust the `fallback_paths` list if your clone lives elsewhere (e.g., when mounting Google Drive).

In [ ]:
%%bash
set -e
REPO_URL="https://github.com/taylor-SUNY-Poly/CS295_Public.git"
TARGET_DIR="/content/CS295_Public"
if [ ! -d "$TARGET_DIR/.git" ]; then
  git clone --depth 1 $REPO_URL $TARGET_DIR
else
  cd $TARGET_DIR
  git pull --ff-only
fi


In [ ]:
from __future__ import annotations
import os
from pathlib import Path

# Identify candidate locations for the colab_port folder.
fallback_paths = [
    Path.cwd(),
    Path('/content') / 'CS295_Public' / 'lunar_lander',
    Path('/content') / 'AI_Projects_Educational' / 'neuroevolution_of_nns' / 'lunar_lander',
    Path('/content') / 'drive' / 'MyDrive' / 'CS295_Public' / 'lunar_lander',
    Path('/content') / 'drive' / 'MyDrive' / 'AI_Projects_Educational' / 'neuroevolution_of_nns' / 'lunar_lander',
]

chosen_root: Path | None = None
for path in fallback_paths:
    candidate = path if path.name == 'colab_port' else path / 'colab_port'
    if candidate.exists() and (candidate / 'lunarlander.py').exists():
        chosen_root = candidate.resolve()
        break

if chosen_root is None:
    raise FileNotFoundError('Could not locate the colab_port folder. Update `fallback_paths` to point to your clone.')

os.chdir(chosen_root)
print(f'Working directory set to: {Path.cwd()}')
print('Available files:')
for item in sorted(Path.cwd().iterdir()):
    print(' -', item.name)


## 3. Inspect or tweak configuration files

Use the helper cells below to open the NEAT config files directly in the notebook. Edit them with `%%writefile` or download/upload via the Colab file browser. Remember to keep notes on which run IDs used which settings, following the lab instructions.

In [ ]:
# Example: preview the Phase 1 configuration
with open('lunarlander_phase1.config') as fh:
    print(fh.read())

## 4. Launch training runs

These cells mirror the command-line usage shown in the student handout. Feel free to duplicate them and adjust phase numbers or add flags (e.g., `--episodes`) as needed. Long-running jobs can be left in the background while you monitor the Colab session.

In [ ]:
%%bash
# Train Phase 1 (edit the command or duplicate the cell for other phases)
python lunarlander.py 1

In [ ]:
%%bash
# Train Phase 2 (uncomment when ready)
# python lunarlander.py 2

In [ ]:
%%bash
# Train Phase 3 (uncomment when ready)
# python lunarlander.py 3

### Evaluate or visualize a saved genome

Point to any existing run ID (e.g., `1.2`) to re-evaluate without retraining, or render a handful of episodes with `--episodes` and `--load`. Use the `--load` flag to skip straight to evaluation/visualisation.

In [ ]:
%%bash
# Example: evaluate the winning genome from run 1.1 without rendering
# python lunarlander.py --load 1.1 --episodes 100

## 5. Review generated artefacts

After a run completes, fitness plots, network diagrams, GIFs, and Markdown reports appear under the corresponding `phase*/` directory. Use the cell below to list the current folder tree.

In [ ]:
%%bash
find . -maxdepth 2 -type f | sort

## 6. Download your results

The export cell bundles any existing `phase1/`, `phase2/`, and `phase3/` directories into a timestamped ZIP archive, then triggers a download in Colab. Run it whenever you want to back up your work or submit artefacts.

In [ ]:
from __future__ import annotations
from datetime import datetime
import zipfile
from pathlib import Path

phase_dirs = [Path(f'phase{i}') for i in range(1, 4)]
existing = [p for p in phase_dirs if p.exists()]

if not existing:
    print('No phase directories found yet. Run some training first!')
else:
    timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    archive_path = Path(f'lunar_lander_artifacts_{timestamp}.zip')
    with zipfile.ZipFile(archive_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for directory in existing:
            for path in directory.rglob('*'):
                if path.is_file():
                    zf.write(path, path.relative_to(Path.cwd()))
    print(f'Created archive: {archive_path}')
    try:
        from google.colab import files
        files.download(str(archive_path))
    except ModuleNotFoundError:
        print('Not running inside Colab; download the ZIP manually.')


---

### Troubleshooting tips

- If `gymnasium[box2d]` fails to install, re-run the installation cell; occasionally Colab backends need an extra retry after the swig package is in place.
- Colab notebooks time out after periods of inactivity. Keep the browser tab active or periodically interact with the notebook to avoid disconnects during long generations.
- Consider enabling Colab Pro (paid) for longer runtimes if you routinely exceed the free-tier limits.